In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import xarray as xr
import os
from typing import TypeVar

import bgk
import bgk.run_params as rp
import bgk.autofigs.util as util

# Helpers

In [ ]:
T = TypeVar("T", float, np.ndarray)

def moving_average(arr: list[float], window: int) -> list[float]:
    ret = np.cumsum(arr)
    ret[window:] = ret[window:] - ret[:-window]
    return ret[window - 1:] / window

def plot(xs: list[float], *ys_argss: list):
    plt.close("all")
    for [ys, *args] in ys_argss:
        if callable(ys):
            ys = [ys(x) for x in xs]
        plt.plot(xs, ys, *args)

def print_errs(pnames: list[str], popts: np.ndarray, pcovs: np.ndarray):
    perrs = np.sqrt(np.diag(pcovs))
    perr_rels = abs(perrs / popts)
    for p, popt, perr_rel in zip(pnames, popts, perr_rels):
        print(f"{p:6s} = {popt:6.3f} ± {100*perr_rel:4.2f}%")


def sigmoid_gaussian(x: T, x0: float, y0: float, sx: float, sy_s: float, sy_g: float) -> T:
    return y0 + sy_s / (1 + np.exp(-(x - x0) / sx)) + sy_g * np.exp(-((x-x0)/sx)**2)

def jac_sigmoid_gaussian(x: T, x0: float, y0: float, sx: float, sy_s: float, sy_g: float) -> T:
    x = (x - x0) / sx
    ex = np.exp(-x)
    ex2 = np.exp(-x**2)

    j_x0 = -1/sx * (sy_s * ex / (1 + ex)**2 - 2 * x * sy_g * ex2)
    j_y0 = np.ones_like(x)
    j_sx = x * j_x0
    j_sy_s = 1 / (1 + ex)
    j_sy_g = ex2

    return np.array([j_x0, j_y0, j_sx, j_sy_s, j_sy_g]).T

def get_sigmoid_gaussian_fit(xs: list[float], ys: list[float]) -> tuple[np.ndarray, np.ndarray]:
    p0 = [xs[np.argmax(ys)], min(ys), max(xs) - min(xs), ys[-1] - ys[0], max(ys) - ys[-1]]
    return opt.curve_fit(sigmoid_gaussian, xs, ys, p0, method="dogbox", bounds=([-np.inf, -np.inf, 0, -np.inf, -np.inf], np.inf))

def get_growth_rate_sigmoid_gaussian(popts: np.ndarray, pcovs: np.ndarray) -> tuple[float, float]:
    return 1 / popts[2], np.sqrt(pcovs[2, 2]) / popts[2]**2


def exponential(x: T, y0: float, sx: float, sy: float) -> T:
    return y0 + sy * np.exp(x / sx)

def get_exponential_fit(xs: list[float], ys: list[float]) -> tuple[np.ndarray, np.ndarray]:
    p0 = [ys[0], xs[-1] - xs[0], ys[-1] - ys[0]]
    return opt.curve_fit(exponential, xs, ys, p0, method='dogbox')

def get_growth_rate_exponential(popts: np.ndarray, pcovs: np.ndarray) -> tuple[float, float]:
    return 1 / popts[1], np.sqrt(pcovs[1, 1]) / popts[1]**2

# Load Data

In [ ]:
path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/exact/B00.25-n512-cont/"

run_manager = bgk.RunManager(path)
params_record = run_manager.params_record
run_diagnostics = run_manager.run_diagnostics

size = run_diagnostics.domain_size
struct_radius = run_diagnostics.hole_radius

wholeSlice = bgk.DataSlice(slice(None, None), "")
centerSlice = bgk.DataSlice(slice(-struct_radius, struct_radius), "Central ")

run_diagnostics.print_params()
run_diagnostics.check_params()

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 201

videoMaker = bgk.VideoMaker(nframes, run_manager)

videoMaker.frame_manager.print_coverage()

In [ ]:
videoMaker.loadData(rp.e_phi)
videoMaker.setSlice(wholeSlice)

# Preprocessing

In [ ]:
rho = 2 * run_diagnostics.hole_radius

In [ ]:
drho = run_diagnostics.domain_size / 100

def getRslice(data: xr.DataArray, rho: float) -> xr.DataArray:
    return data.where((rho <= videoMaker.grid_rho) & (videoMaker.grid_rho < rho + drho))

def get_y(data: xr.DataArray, rho: float) -> float:
    return abs(getRslice(data, rho)).mean() * 1e5

In [ ]:
%matplotlib inline
plt.close("all")
im = plt.imshow(
            getRslice(videoMaker.slicedDatas[-1], rho),
            origin="lower",
            extent=(
                videoMaker._currentSlice.slice.start,
                videoMaker._currentSlice.slice.stop,
                videoMaker._currentSlice.slice.start,
                videoMaker._currentSlice.slice.stop,
            ),)
plt.colorbar(im)

In [ ]:
rolling_window = 10
ts = moving_average(videoMaker.times, rolling_window)
ys = moving_average([get_y(data, rho) for data in videoMaker.slicedDatas], rolling_window)

# Fits

## Fit Sigmoid

In [ ]:
popts_sg, pcovs_sg = get_sigmoid_gaussian_fit(ts, ys)
plot(ts, [ys, "."], [sigmoid_gaussian(ts, *popts_sg), "-"])
print_errs(["t0", "y0", "st", "sy_s", "sy_g"], popts_sg, pcovs_sg)

## Fit Exponential

In [ ]:
tstart = 0
tstop = 50
growth_phase = slice(np.argmax(ts > tstart), np.argmax(ts > tstop))

ts2 = ts[growth_phase]
ys2 = ys[growth_phase]

In [ ]:
popts_exp, pcovs_exp = get_exponential_fit(ts2, ys2)
print_errs(["y0", "st", "sy"], popts_exp, pcovs_exp)
plot(ts2, [ys2, "."], [exponential(ts2, *popts_exp), "-"], [exponential(ts2 - popts_sg[0], *popts_sg[1:4]), "--"])
# plot(ts2, [ys2, "."], [exponential(ts2, *popts_exp), "-"])

In [ ]:
growth_rate_exp, growth_rate_exp_err = get_growth_rate_exponential(popts_exp, pcovs_exp)
growth_rate_sg, growth_rate_sg_err = get_growth_rate_sigmoid_gaussian(popts_sg, pcovs_sg)

print(f"growth rate exp: {growth_rate_exp:.4f} ± {growth_rate_exp_err:.4f}")
print(f"growth rate sg:  {growth_rate_sg:.4f} ± {growth_rate_sg_err:.4f}")

## Semilog

In [ ]:
# y0 = popts_exp[0]
y0 = popts_sg[1]
# y0 = ys[0]

In [ ]:
tstart = 23
tstop = 40
linear_phase = slice(np.argmax(ts > tstart), np.argmax(ts > tstop))

ts3 = ts[linear_phase]
ys3 = np.log(ys[linear_phase] - y0)

wherefinite = np.isfinite(ys3) 
ts3 = ts3[wherefinite]
ys3 = ys3[wherefinite]

In [ ]:
def linear(t: float, m: float, b: float) -> float:
    return m*t + b
    
popts_lin, pcovs_lin = opt.curve_fit(linear, ts3, ys3)
print_errs(["m", "b"], popts_lin, pcovs_lin)

In [ ]:
plt.close("all")
fig, ax = plt.subplots(1,1)
ax.plot(ts, ys - y0, label="From run")
ax.plot(ts3, np.exp(linear(ts3, *popts_lin)), label=rf"Linear fit ($\omega_I={popts_lin[0]:.3f} \pm {np.sqrt(pcovs_lin[0,0]):.3f}$)")
# ax.plot(ts, exponential(ts, *popts_exp) - y0, label="exp")
ax.plot(ts, sigmoid_gaussian(ts, *popts_sg) - y0, label="sg")
ax.axvline(ts3[0], color="grey", linestyle="--")
ax.axvline(ts3[-1], color="grey", linestyle="--")
ax.set_xlabel("Time")
ax.set_ylabel(rf"$\propto const + \int|E_\phi(\rho,\phi)| d\phi$")
ax.set_yscale('log')
ax.set_title(rf"Growth of Instability in {videoMaker._currentParam.title} ($B_0={params_record.B0}$, $\rho={rStart:.3f}$, {params_record.init_strategy.capitalize()})")
ax.legend()
plt.show()

In [ ]:
os.makedirs("figs-manual", exist_ok=True)
# util.save_fig(fig, f"figs-manual/growth-{videoMaker._currentParam.name}-{params_record.init_strategy}-B{params_record.B0:4.2f}-n{params_record.res}.png")

## Rho Dependence

In [ ]:
from dataclasses import dataclass

def get_ys(rho: float) -> np.ndarray:
    return moving_average([get_y(data, rho) for data in videoMaker.slicedDatas], rolling_window)
    
@dataclass
class Growth:
    time_start: float
    time_stop: float
    rate: float
    rate_err: float

    def exclude(self) -> bool:
        return np.nan in [self.time_start, self.time_stop, self.rate, self.rate_err] or self.rate < self.rate_err or self.rate_err < 1e-6

def get_growth(ts: np.ndarray, ys: np.ndarray) -> Growth:
    lbound = 0
    ubound = np.argmax(ys) // 2

    try:
        popts, pcovs = get_exponential_fit(ts[lbound:ubound], ys[lbound:ubound])
        rate, rate_err = get_growth_rate_exponential(popts, pcovs)
        return Growth(ts[lbound], ts[ubound], rate, rate_err)
    except:
        return Growth(np.nan, np.nan, np.nan, np.nan)


In [ ]:
rhos = np.linspace(run_diagnostics.hole_radius / 2, run_diagnostics.domain_size / 2, 100)
yss = [get_ys(rho) for rho in rhos]

In [ ]:
# grs_errs_of_rho = [get_growth_rate_exponential(*get_exponential_fit(ts[growth_phase], ys[growth_phase])) for ys, growth_phase in zip(ys_of_rho[:nrhos], growth_phases[:nrhos])]
growths = [get_growth(ts, ys) for ys in yss]

In [ ]:
for i, g in enumerate(growths):
    if g.exclude():
        rhos[i] = np.nan

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.set_zorder(ax2.get_zorder() + 1)
ax1.set_facecolor('none')

color_ax1 = "b"
color_ax2 = "g"

ax1.set_title("Linear Growth Rates")
ax1.set_xlabel(rf"$\rho$")
ax1.set_ylabel("Growth Rate", color=color_ax1)
ax1.errorbar(rhos, [g.rate for g in growths], yerr=[g.rate_err for g in growths], color=color_ax1, capsize=1.7)
ax2.set_ylabel("Linear Growth Phase", color=color_ax2)
ax2.vlines(rhos, [g.time_start for g in growths], [g.time_stop for g in growths], colors=color_ax2, alpha=.4)


In [ ]:
os.makedirs("figs-test", exist_ok=True)
util.save_fig(fig, f"figs-test/growth1-{videoMaker._currentParam.name}-{params_record.init_strategy}-B{params_record.B0:4.2f}-n{params_record.res}.png")

## Detecting Linear Region

In [ ]:
logys = np.log(ys - y0)
dlogys = np.gradient(logys, ts)
ddlogys = np.gradient(dlogys, ts)

In [ ]:
plot(ts, [ddlogys])

In [ ]:
smooth_width = 30
x_conv = np.linspace(-3,3,smooth_width)
y_conv = (4*x_conv**2 - 2) * np.exp(-x_conv**2) / smooth_width * 8

In [ ]:
ddlogys_conv = np.convolve(logys, y_conv, mode="same")
plot(ts, [ddlogys_conv])